# 📰 News Sentiment Analyzer

Analyze financial news sentiment and extract future-looking insights using rule-based methods.

[Deep Charts YouTube Channel](https://www.youtube.com/@DeepCharts)

## Installation and Setup

In [36]:
# Install required packages
# !pip install -q pandas gnews ipywidgets plotly textblob vaderSentiment

In [37]:
# Import required libraries
import pandas as pd
from gnews import GNews
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
from datetime import datetime, timedelta

## Create User Interface

In [38]:
# Create widgets for user input
ticker_input = widgets.Text(
    value='AAPL',
    placeholder='Enter stock ticker',
    description='Ticker:',
    disabled=False
)

num_articles_slider = widgets.IntSlider(
    value=6,
    min=1,
    max=20,
    step=1,
    description='Articles:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

analyze_button = widgets.Button(
    description='Analyze News',
    disabled=False,
    button_style='primary',
    tooltip='Click to analyze news'
)

# Display the input widgets
print("News Sentiment Analyzer Configuration")
display(ticker_input)
display(num_articles_slider)
display(analyze_button)

News Sentiment Analyzer Configuration


Text(value='UOGU.L', description='Ticker:', placeholder='Enter stock ticker')

IntSlider(value=11, continuous_update=False, description='Articles:', max=20, min=1)

Button(button_style='primary', description='Analyze News', style=ButtonStyle(), tooltip='Click to analyze news…

Fetching 11 news articles about UOGU.L...
No news articles found for this ticker


## Sentiment Analysis Functions

In [39]:
# Initialize sentiment analyzer
vader_analyzer = SentimentIntensityAnalyzer()

# Keywords for future-looking analysis
FUTURE_KEYWORDS = [
    'forecast', 'outlook', 'predict', 'expect', 'project', 'estimate',
    'will', 'going to', 'plan', 'target', 'guidance', 'future',
    'next quarter', 'next year', 'upcoming', 'ahead', 'potential',
    'growth', 'expansion', 'acquisition', 'merger', 'dividend', 'buyback'
]

# Positive and negative keywords for sentiment analysis
POSITIVE_KEYWORDS = [
    'profit', 'gain', 'growth', 'increase', 'rise', 'surge', 'jump',
    'beat', 'exceed', 'outperform', 'success', 'win', 'positive',
    'strong', 'robust', 'optimistic', 'bullish', 'upgrade', 'buy'
]

NEGATIVE_KEYWORDS = [
    'loss', 'decline', 'drop', 'fall', 'plunge', 'slump', 'miss',
    'disappoint', 'underperform', 'failure', 'negative', 'weak',
    'concern', 'risk', 'warn', 'cut', 'downgrade', 'sell', 'bearish'
]

def analyze_sentiment_textblob(text):
    """Analyze sentiment using TextBlob"""
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity

    if polarity > 0.1:
        return 'positive'
    elif polarity < -0.1:
        return 'negative'
    else:
        return 'neutral'

def analyze_sentiment_vader(text):
    """Analyze sentiment using VADER"""
    scores = vader_analyzer.polarity_scores(text)

    if scores['compound'] >= 0.05:
        return 'positive'
    elif scores['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

def analyze_sentiment_keywords(text):
    """Analyze sentiment based on keyword matching"""
    text_lower = text.lower()

    positive_count = sum(1 for keyword in POSITIVE_KEYWORDS if keyword in text_lower)
    negative_count = sum(1 for keyword in NEGATIVE_KEYWORDS if keyword in text_lower)

    if positive_count > negative_count:
        return 'positive'
    elif negative_count > positive_count:
        return 'negative'
    else:
        return 'neutral'

def is_future_looking(text):
    """Determine if text contains future-looking statements"""
    text_lower = text.lower()

    # Check for future keywords
    future_keyword_count = sum(1 for keyword in FUTURE_KEYWORDS if keyword in text_lower)

    # Check for financial forward-looking phrases
    future_phrases = [
        r'q[1-4]\s*\d{4}',  # Q1 2023, Q2 2024, etc.
        r'\d{4}\s*outlook',
        r'forecast\s*for\s*\d{4}',
        r'next\s*(year|quarter|month)',
        r'expected\s*to',
        r'projected\s*to',
        r'estimated\s*to'
    ]

    future_phrase_count = sum(1 for phrase in future_phrases if re.search(phrase, text_lower))

    return (future_keyword_count + future_phrase_count) > 0

def analyze_sentiment_combined(text):
    """Combine multiple sentiment analysis methods"""
    methods = [
        analyze_sentiment_textblob(text),
        analyze_sentiment_vader(text),
        analyze_sentiment_keywords(text)
    ]

    # Return the most common sentiment
    if methods.count('positive') >= 2:
        return 'positive'
    elif methods.count('negative') >= 2:
        return 'negative'
    else:
        return 'neutral'

## Core Functions

In [40]:
# Function to fetch news
def fetch_news(ticker, num_articles):
    # Initialize GNews client
    google_news = GNews()
    google_news.period = '7d'  # Last 7 days
    google_news.max_results = num_articles

    print(f"Fetching {num_articles} news articles about {ticker}...")

    try:
        # Fetch news articles
        news = google_news.get_news(f"{ticker} stock")

        if not news:
            print("No news articles found for this ticker")
            return None

        # Extract news data
        news_data = []
        for article in news[:num_articles]:
            news_data.append({
                'title': article.get('title', ''),
                'description': article.get('description', ''),
                'published_date': article.get('published date', ''),
                'publisher': article.get('publisher', {}).get('title', '')
            })

        # Display raw news titles
        print("\nRaw News Headlines:")
        for i, article in enumerate(news_data, 1):
            print(f"{i}. {article['title']}")

        return news_data

    except Exception as e:
        print(f"Error fetching news: {str(e)}")
        return None

In [41]:
# Analysis function
def perform_sentiment_analysis(news_data):
    results = []

    print(f"\nAnalyzing {len(news_data)} articles...")

    for i, article in enumerate(news_data):
        print(f"Analyzing article {i + 1} of {len(news_data)}...")

        try:
            # Combine title and description for analysis
            text = f"{article['title']}. {article['description']}"

            # Analyze sentiment
            sentiment = analyze_sentiment_combined(text)

            # Check if future-looking
            future_looking = is_future_looking(text)

            # Calculate confidence score
            vader_score = vader_analyzer.polarity_scores(text)['compound']
            textblob_score = TextBlob(text).sentiment.polarity
            confidence = (abs(vader_score) + abs(textblob_score)) / 2

            # Store results
            results.append({
                'Title': article['title'],
                'Publisher': article['publisher'],
                'Date': article['published_date'],
                'Sentiment': sentiment,
                'Future Looking': future_looking,
                'Confidence': f"{confidence:.2f}"
            })

        except Exception as e:
            print(f"Failed to analyze article: {article['title']}\nError: {str(e)}")
            continue

    return pd.DataFrame(results)

In [42]:
# Function to display results
def display_results(df):
    if df.empty:
        print("No valid analysis results were returned")
        return

    print("\nAnalysis Results:")

    # Show summary metrics
    positive_count = len(df[df['Sentiment'] == 'positive'])
    negative_count = len(df[df['Sentiment'] == 'negative'])
    neutral_count = len(df[df['Sentiment'] == 'neutral'])
    future_count = len(df[df['Future Looking'] == True])
    total_count = len(df)

    print(f"Positive Sentiment: {positive_count}/{total_count}")
    print(f"Negative Sentiment: {negative_count}/{total_count}")
    print(f"Neutral Sentiment: {neutral_count}/{total_count}")
    print(f"Future-Looking Insights: {future_count}/{total_count}")

    # Calculate overall sentiment score
    sentiment_score = (positive_count - negative_count) / total_count
    print(f"Overall Sentiment Score: {sentiment_score:.2f}")

    # Show full results table
    display(df)

    # Create visualizations
    # Sentiment Distribution
    sentiment_counts = df['Sentiment'].value_counts()
    fig1 = px.pie(values=sentiment_counts.values, names=sentiment_counts.index,
                 title="Sentiment Distribution")

    # Future-Looking Insights
    future_counts = df['Future Looking'].value_counts()
    fig2 = px.pie(values=future_counts.values, names=[str(val) for val in future_counts.index],
                 title="Future-Looking Insights")

    # Sentiment by Publisher
    if len(df['Publisher'].unique()) > 1:
        publisher_sentiment = df.groupby(['Publisher', 'Sentiment']).size().unstack(fill_value=0)
        fig3 = px.bar(publisher_sentiment, title="Sentiment by Publisher",
                     labels={'value': 'Count', 'Publisher': 'News Source'})
    else:
        fig3 = None

    # Display plots
    fig1.show()
    fig2.show()
    if fig3:
        fig3.show()

## Event Handling

In [43]:
# Event handler for the analyze button
def on_analyze_button_clicked(b):
    # Clear previous output
    clear_output(wait=True)

    # Redisplay the input widgets
    print("News Sentiment Analyzer Configuration")
    display(ticker_input)
    display(num_articles_slider)
    display(analyze_button)

    # Get values from widgets
    ticker = ticker_input.value.upper()
    num_articles = num_articles_slider.value

    # Fetch news
    news_data = fetch_news(ticker, num_articles)

    if news_data:
        # Perform analysis
        df = perform_sentiment_analysis(news_data)

        if not df.empty:
            # Display results
            display_results(df)
        else:
            print("No valid analysis results were returned")

# Attach the event handler to the button
analyze_button.on_click(on_analyze_button_clicked)

## Usage Instructions

1. Run all cells to initialize the application
2. Enter a stock ticker and select the number of articles
3. Click the "Analyze News" button
4. View the results in the output area below the button

## How It Works:

- **News Fetching**: Uses GNews API to retrieve recent financial news
- **Sentiment Analysis**: Combines three methods:
  1. TextBlob - Rule-based sentiment analysis
  2. VADER - Valence Aware Dictionary and sEntiment Reasoner
  3. Keyword matching - Custom financial keywords
- **Future-Looking Detection**: Identifies forward-looking statements using financial keywords and patterns
- **Visualization**: Creates interactive charts to visualize sentiment distribution

This implementation provides a robust, rule-based alternative to LLM-based sentiment analysis.